In [5]:
class knauf():
    # Constructor و سایر متدهای اصلی تغییری نمی‌کنند
    # ...

    def concat_data(self):
        if not hasattr(self, 'cal_data') or not hasattr(self, 'read_list'):
            raise ValueError("DataFrames are missing. Ensure to call 'import_data' and 'create_dataframe_from_data' first.")
        self.knauf_list = pd.concat([self.read_list, self.cal_data[['amount_metr']]], axis=1)

    def add_salable_amount(self):
        divisors = [4, 4, 4, 3, 1000, 1000, 100, 1, 1, 1, 1, 1]
        if len(self.cal_data['amount_metr']) != len(divisors):
            raise ValueError("Length of divisors does not match length of amount_metr.")
        self.cal_data['salable_amount'] = [
            self.find_smallest_divisible(amount, divisor)
            for amount, divisor in zip(self.cal_data['amount_metr'], divisors)
        ]

    def calculate_mat_price(self):
        if 'salable_amount' not in self.cal_data or 'mat_price' not in self.read_list:
            raise ValueError("Missing necessary columns for price calculation.")
        self.knauf_list['material_price'] = self.cal_data['salable_amount'] * self.read_list['mat_price']

    def add_sale_percentages(self):
        if 'material_price' not in self.knauf_list:
            raise ValueError("Material price not calculated yet.")
        sum_total_price = self.knauf_list['material_price'].sum()
        self.knauf_list.loc[len(self.knauf_list)] = ['Total', '', '', '', sum_total_price]
        
        # بسته‌بندی
        for i, row in self.knauf_list.iterrows():
            if 'panel_metr' in row['Materials']:
                self.knauf_list.loc[i, 'adjusted_price'] = row['material_price'] * 1.035
            elif row['Materials'] in ['nabshi_metr', 'f_47_metr', 'u_36_metr']:
                self.knauf_list.loc[i, 'adjusted_price'] = row['material_price'] * 1.025
        
        # انبارداری
        storage_price = self.knauf_list['adjusted_price'].sum() * 0.01
        self.knauf_list.loc[len(self.knauf_list)] = ['Storage Fee', '', '', '', storage_price]
        
        # مالیات بر ارزش افزوده
        tax = (sum_total_price + storage_price) * 0.1
        self.knauf_list.loc[len(self.knauf_list)] = ['VAT', '', '', '', tax]
        
    def save_to_excel(self):
        if not hasattr(self, 'knauf_list'):
            raise ValueError("The knauf_list has not been initialized. Please run concat_data first.")
        file_name = f"{self.name_p}.xlsx"
        self.knauf_list.to_excel(file_name, index=False)
        print(f'Data for material saved to {file_name}')
